In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import time
from pathlib import Path
import os
import openpyxl
from sec_cik_mapper import StockMapper
from datetime import datetime, timedelta
from urllib.parse import urljoin
from alpha_vantage.timeseries import TimeSeries

In [3]:
# Get the current working directory
current_dir = os.getcwd()

# Resolve the parent directories
project_general_path = Path(current_dir).resolve()
print(project_general_path)

C:\Users\Adam Krupa\OneDrive\Pulpit\Investing\earnings-report-analysis


In [4]:
def get_tickers():
    # URL of the website containing the S&P 500 tickers
    url = 'https://www.slickcharts.com/sp500'

    # Fetch the page content with headers to avoid being blocked
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'table table-hover table-borderless table-sm'})
        tickers = []
        
        # Extract tickers from the table
        if table:
            for row in table.find('tbody').find_all('tr'):
                columns = row.find_all('td')
                ticker = columns[2].text.strip()  # The third column contains the ticker symbol
                tickers.append(ticker)
        else:
            print("Table not found in the page")
    else:
        print("Failed to fetch S&P 500 tickers")
        tickers = []

    return tickers
    


In [5]:
def get_dates(tickers):
    # Initialize an empty DataFrame to store valid earnings data
    data = pd.DataFrame(columns=['Ticker', 'Date'])

    # Loop through each stock ticker
    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        # Fetch the earnings history (past earnings reports)
        earnings_history = ticker.earnings_dates
        
        # Skip if no earnings data is available
        if earnings_history is None:
            print(f"{ticker_symbol}: possibly delisted; no earnings dates found")
            continue

        # Iterate through each earnings report and determine the correct date classification
        for index, row in earnings_history.iterrows():
            report_time = pd.to_datetime(row.name)  # Access the index (which is the earnings date) and convert it to datetime
            # if report_time.hour >= 16:  # After 4 PM, classify as next day because the trade based on this knowledge can effectively only be executed the day after
            #     adjusted_date = (report_time + pd.Timedelta(days=1)).date()
            # else:  # Otherwise, use the same day
            #     adjusted_date = report_time.date()

            # Add each adjusted earnings date as a separate row in the DataFrame
            new_row = {'Ticker': ticker_symbol, 'Date': report_time}
            data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

    return data


In [6]:
def fetch_stock_prices(data, api_key):
    # Initialize the TimeSeries object with your API key
    ts = TimeSeries(key=api_key, output_format='pandas')

    # Function to fetch and filter stock prices for a given ticker and date
    def get_prices(ticker, target_date):
        # Fetch daily stock prices
        try:
            stock_data, _ = ts.get_daily(symbol=ticker, outputsize='full')
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            return {}

        # Ensure the index is in datetime format
        stock_data.index = pd.to_datetime(stock_data.index)

        # Define the date range
        start_date = target_date - timedelta(days=60)
        end_date = target_date + timedelta(days=30)

        # Filter the data for the specified date range
        filtered_data = stock_data.loc[start_date:end_date]

        # Convert the filtered data to a dictionary with dates as keys and closing prices as values
        price_dict = filtered_data['4. close'].to_dict()

        return price_dict

    # Apply the function to each row in the DataFrame
    data['Prices'] = data.apply(lambda row: get_prices(row['Ticker'], pd.to_datetime(row['Date'])), axis=1)

    return data

In [19]:
def get_stock_prices_alpha_vantage(data, api_key):
    # Initialize the TimeSeries object with your API key
    ts = TimeSeries(key=api_key, output_format='pandas')

    # Function to fetch and filter stock prices for a given ticker and date
    def get_prices(ticker, target_date):
        # Ensure target_date is timezone-naive
        target_date = pd.to_datetime(target_date).tz_localize(None)

        # Fetch daily stock prices
        try:
            stock_data, _ = ts.get_daily(symbol=ticker, outputsize='full')
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            return {}

        # Ensure the index is in datetime format and timezone-naive
        stock_data.index = pd.to_datetime(stock_data.index).tz_localize(None)

        # Define the date range
        start_date = target_date - timedelta(days=60)
        end_date = target_date + timedelta(days=30)

        # Filter the data for the specified date range
        filtered_data = stock_data.loc[start_date:end_date]

        # Convert the filtered data to a dictionary with dates as keys and closing prices as values
        price_dict = filtered_data['4. close'].to_dict()

        return price_dict

    # Apply the function to each row in the DataFrame
    data['Prices'] = data.apply(
        lambda row: get_prices(row['Ticker'], row['Date']), axis=1
    )

    return data

In [28]:
def get_stock_prices_yahoo(data):
    # Function to fetch and filter stock prices for a given ticker and date
    def get_prices(ticker, target_date):
        # Ensure target_date is in datetime format
        target_date = pd.to_datetime(target_date)

        # Define the date range
        start_date = target_date - timedelta(days=60)
        end_date = target_date + timedelta(days=30)

        # Fetch historical stock prices using Yahoo Finance
        try:
            stock_data = yf.download(ticker, start=start_date, end=end_date).round(3)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            return {}

        # Convert the filtered data to a dictionary with dates as keys and closing prices as values
        price_dict = stock_data['Close'].to_dict()

        return price_dict

    # Apply the function to each row in the DataFrame
    data['Prices'] = data.apply(
        lambda row: get_prices(row['Ticker'], row['Date']), axis=1
    )

    return data

In [21]:
tickers = get_tickers()

In [22]:
tickers_short = tickers[30:40]

In [23]:
tickers_short

['WFC', 'NOW', 'ACN', 'MCD', 'PEP', 'IBM', 'DIS', 'AMD', 'LIN', 'TMO']

In [24]:
data = data = get_dates(tickers_short)

C:\Users\Adam Krupa\AppData\Local\Temp\ipykernel_132260\2737740090.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)


In [29]:

data = get_stock_prices_yahoo(data)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['WFC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2025-11-15 19:00:00-05:00 -> 2026-02-13 19:00:00-05:00) (Yahoo error = "Data doesn\'t exist for startDate = 1763251200, endDate = 1771027200")')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['WFC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2025-08-15 20:00:00-04:00 -> 2025-11-13 19:00:00-05:00) (Yahoo error = "Data doesn\'t exist for startDate = 1755302400, endDate = 1763078400")')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['WFC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2025-05-15 20:00:00-04:00 -> 2025-08-13 20:00:00-04:00) (Yahoo error = "Data doesn\'t exist for startDate = 1747353600, endDate = 1755129600")')
[*********************100%**************

In [31]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

data['Prices']

0                                            {'WFC': {}}
1                                            {'WFC': {}}
2                                            {'WFC': {}}
3                                            {'WFC': {}}
4      {'WFC': {2024-11-18 00:00:00: 74.09, 2024-11-1...
5      {'WFC': {2024-08-12 00:00:00: 51.762, 2024-08-...
6      {'WFC': {2024-05-13 00:00:00: 60.447, 2024-05-...
7      {'WFC': {2024-02-12 00:00:00: 47.995, 2024-02-...
8      {'WFC': {2023-11-13 00:00:00: 39.72, 2023-11-1...
9      {'WFC': {2023-08-14 00:00:00: 42.237, 2023-08-...
10     {'WFC': {2023-05-15 00:00:00: 37.147, 2023-05-...
11     {'WFC': {2023-02-13 00:00:00: 45.758, 2023-02-...
12                                           {'NOW': {}}
13                                           {'NOW': {}}
14                                           {'NOW': {}}
15     {'NOW': {2024-11-25 00:00:00: 1052.71, 2024-11...
16     {'NOW': {2024-08-26 00:00:00: 827.86, 2024-08-...
17     {'NOW': {2024-05-28 00:0